In [3]:
import transformers
import torch
import pandas as pd
import os

/home/charaf/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print(os.getcwd())

/home/charaf/DeepLearning/Project/Deep_Learning


In [5]:
## Read data/paradox.tsv file as pandas dataframe
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, 'data')
df = pd.read_csv(data_dir +'/paradetox.tsv', sep='\t')
df.head(5)


,toxic,neutral1,neutral2,neutral3
0,he had steel balls too !,he was brave too!,NaN,NaN
1,"dude should have been taken to api , he would ...",It would have been good if he went to api. He ...,NaN,NaN
2,"im not gonna sell the fucking picture , i just...","I'm not gonna sell the picture, i just want to...",NaN,NaN
3,the garbage that is being created by cnn and o...,the news that is being created by cnn and othe...,The news that is being created by cnn and othe...,the garbage that is being created by cnn and o...
4,the reason they dont exist is because neither ...,The reason they don't exist is because neither...,NaN,NaN


### Kept for every toxic sentence just one non-toxic

In [7]:
## get NaN values per column
df.isna().sum()

toxic           0
neutral1        0
neutral2     5973
neutral3    10065
dtype: int64

In [8]:
## rename neutral1 column to neutral
df.rename(columns={'neutral1': 'neutral'}, inplace=True)

In [9]:
filtered_df = df[["toxic", "neutral"]]

In [10]:
df_dict = filtered_df.to_dict(orient="records")

In [11]:
from datasets import load_dataset, Dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments

In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, EarlyStoppingCallback


In [ ]:
from trl import SFTTrainer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, EarlyStoppingCallback, AutoModelForCausalLM

# Model
model_name = "Qwen/Qwen2.5-3B-Instruct"
base_tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.29s/it]


In [15]:
def detoxify_sentence(sentence: str, max_new_tokens: int = 100) -> str:
    # Define chat-style prompt
    messages = [
        {
            "role": "system",
            "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."
        },
        {
            "role": "user",
            "content": f'Detoxify this sentence: "{sentence}"'
        }
    ]

    # Format the input using Qwen's chat template
    prompt_text = base_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Tokenize and move to the same device as the model
    inputs = base_tokenizer([prompt_text], return_tensors="pt").to(base_model.device)

    # Generate output
    with torch.no_grad():
        outputs = base_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False  # greedy decoding for consistency
        )

    # Remove the prompt part and decode only the newly generated tokens
    generated_ids = outputs[0][inputs.input_ids.shape[1]:]
    response = base_tokenizer.decode(generated_ids, skip_special_tokens=True)

    return response.strip()

#%%
toxic = "delete the page and shut up"
cleaned = detoxify_sentence(toxic)
print("Detoxified Output:", cleaned)

/home/charaf/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/charaf/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/charaf/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Detoxified Output: I'm sorry, but that sentence is not appropriate or constructive. Perhaps you could rephrase it in a more positive way, such as "Let's delete the page and move on." If you need help with a different task or have another sentence you'd like me to assist with, feel free to let me know!


In [20]:
def preprocess(example):
    input_text = f"detoxify: {example['toxic']}"
    target_text = example["neutral"]
    input_enc = tokenizer(input_text, truncation=True, padding="max_length", max_length=64)
    target_enc = tokenizer(target_text, truncation=True, padding="max_length", max_length=64)

    input_enc["labels"] = target_enc["input_ids"]
    return input_enc

tokenized_dataset = dataset.map(preprocess)
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

training_args = TrainingArguments(
    output_dir="./detoxifier",
    per_device_train_batch_size=4,
    num_train_epochs=10, # Set a reasonably high number of epochs, early stopping will stop it
    save_total_limit=1,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="steps", # Evaluate every eval_steps or logging_steps if eval_steps is not set
    # evaluation_strategy="epoch", # Alternatively, evaluate at the end of each epoch
    eval_steps=50, # Evaluate every 50 steps (adjust as needed) - or remove to use logging_steps
    load_best_model_at_end=True, # Load the best model based on the monitored metric
    metric_for_best_model="eval_loss", # Monitor validation loss for early stopping
    greater_is_better=False # For loss, lower is better
)

# Define the EarlyStoppingCallback
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3, # Stop after 3 evaluations without improvement
    early_stopping_threshold=0.0, # Optional: minimum improvement to consider as improvement
)

# Initialize the SFTTrainer with the callback and evaluation dataset
trainer = SFTTrainer(
    model=model,
    #tokenizer=tokenizer, # Keep if you need the trainer to handle tokenization
    args=training_args,
    train_dataset=train_dataset, # Use the training split
    eval_dataset=eval_dataset,   # Add the evaluation split
    formatting_func=None, # optional if dataset is already tokenized
    callbacks=[early_stopping_callback], # Add the early stopping callback
)

# Start training
trainer.train()


NameError: name 'dataset' is not defined

In [31]:
trainer.model.save_pretrained("models/sft-detoxifier")
trainer.tokenizer.save_pretrained("models/sft-detoxifier")

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


('models/sft-detoxifier/tokenizer_config.json',
 'models/sft-detoxifier/special_tokens_map.json',
 'models/sft-detoxifier/spiece.model',
 'models/sft-detoxifier/added_tokens.json',
 'models/sft-detoxifier/tokenizer.json')

In [36]:
## Load the model and tokenizer from models
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "models/sft-detoxifier"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# Test the model


In [ ]:
def detoxify(text, model=model, tokenizer=tokenizer):
    inputs = tokenizer(f"toxic: {text}. neutral: ", return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=120)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(detoxify("You're such an idiot"))

detoxify: You're such an idiot


In [51]:
detoxify("You're such an idiot", model=base_model, tokenizer=base_tokenizer)

"detoxify: You're such an idiot"

In [5]:
from transformers import AutoModelForCausalLM

In [29]:
os.getcwd()

'/home/charaf/DeepLearning/Project/Deep_Learning'

In [30]:
model_name

'./models/sft-detoxifier/checkpoint-23500'

In [ ]:
## Load detoxifier last checkpoint from models/detoxifier/checkpoint-23500
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "detoxifier/checkpoint-23500"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
# Test the model
def detoxify(text, model=model, tokenizer=tokenizer):
    inputs = tokenizer(f"toxic: {text}. neutral: ", return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=120)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)
print(detoxify("You're such an idiot"))

toxic: You're such an idiot. neutral:
